In [ ]:
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
from cytoolz import compose
import operator as op

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell

from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_rss
import matplotlib.pyplot as plt
import matplotlib as mpl
from adjustText import adjust_text
import seaborn as sns
from pyscenic.binarization import binarize

In [ ]:
# Helper function
def derive_regulons(folder):
    # Load enriched motifs.
    motifs = load_motifs(folder+'motifs.csv')
    motifs.columns = motifs.columns.droplevel(0)

    def contains(*elems):
        def f(context):
            return any(elem in context for elem in elems)
        return f

    # For the creation of regulons we only keep the 10-species databases and the activating modules. We also remove the
    # enriched motifs for the modules that were created using the method 'weight>50.0%' (because these modules are not part
    # of the default settings of modules_from_adjacencies anymore.
    motifs = motifs[
        np.fromiter(map(compose(op.not_, contains('weight>50.0%')), motifs.Context), dtype=np.bool) & \
        np.fromiter(map(contains('mm9-tss-centered-10kb-10species.mc9nr', 
                                 'mm9-500bp-upstream-10species.mc9nr', 
                                 'mm9-tss-centered-5kb-10species.mc9nr'), motifs.Context), dtype=np.bool) & \
        np.fromiter(map(contains('activating'), motifs.Context), dtype=np.bool)]

    # We build regulons only using enriched motifs with a NES of 3.0 or higher; we take only directly annotated TFs or TF annotated
    # for an orthologous gene into account; and we only keep regulons with at least 10 genes.
    regulons = list(filter(lambda r: len(r) >= 10, df2regulons(motifs[(motifs['NES'] >= 3.0) 
                                                                      & ((motifs['Annotation'] == 'gene is directly annotated')
                                                                        | (motifs['Annotation'].str.startswith('gene is orthologous to')
                                                                           & motifs['Annotation'].str.endswith('which is directly annotated for motif')))
                                                                     ])))
    
    # Rename regulons, i.e. remove suffix.
    regulons = list(map(lambda r: r.rename(r.transcription_factor), regulons))

    # Pickle these regulons.
    with open(folder+'regulons.dat', 'wb') as f:
        pickle.dump(regulons, f)

# SCENIC

### Load the expression matrix

In [ ]:
SC_EXP_FNAME = "Data/Cells/CD4/cd4.matrix.csv"
ex_matrix = pd.read_csv(SC_EXP_FNAME, sep=' ', header=0, index_col=0).T
ex_matrix.index = ex_matrix.index.str.replace('.', '-')
ex_matrix

__METADATA CLEANING__

In [ ]:
BASE_FOLDER = "Data/Cells/CD4/"
df_cluster = pd.read_csv(BASE_FOLDER+"cd4.clusters.csv")
df_tumor = pd.read_csv(BASE_FOLDER+"cd4.tumor.csv")
df_cluster['x'] = pd.Series(df_cluster['x'], dtype="category")
df_metadata = df_cluster.merge(df_tumor, on="Unnamed: 0")
df_metadata.columns = ["Cell ID", "Cluster", "Tumor"]
df_metadata = df_metadata.set_index("Cell ID")
df_metadata.to_csv(BASE_FOLDER+"SCENIC/cd4.metadata.csv", index=False)
df_metadata

In [ ]:
# Regulons
BASE_FOLDER = "Data/Cells/CD4/SCENIC/"
derive_regulons(BASE_FOLDER)

with open(BASE_FOLDER+'regulons.dat', 'rb') as f:
    regulons = pickle.load(f)

In [ ]:
BASE_FOLDER = "Data/Cells/CD4/SCENIC/"
REGULONS_DAT_FNAME = os.path.join(BASE_FOLDER, 'regulons.dat')
with open(REGULONS_DAT_FNAME, 'rb') as f:
    regulons = pickle.load(f)
regulon_df = []
for i,regulon in enumerate(regulons):
    gene2weight = pd.Series(dict(regulon.gene2weight)).sort_values(ascending=False).round(2)
    regulon_df.append([regulon.name, len(regulon.genes), regulon.score, list(zip(gene2weight.index, gene2weight))])
regulon_df = pd.DataFrame(regulon_df, columns=["Regulon", "Size", "Score", "Genes"]).sort_values(by="Score", ascending=False).set_index('Regulon')
regulon_df.to_excel(BASE_FOLDER+"regulon_df.xlsx")
regulon_df

In [ ]:
# Calculate AUC Matrix
auc_mtx = aucell(ex_matrix, regulons, seed=42)
auc_mtx.to_csv(BASE_FOLDER+"auc.csv")
auc_mtx

In [ ]:
# %%capture
# from tqdm import tqdm_notebook as tqdm
# tqdm().pandas()

# for start in tqdm(range(0, len(auc_mtx.columns))):
#     print(auc_mtx.iloc[:,start:start+1].columns)
#     binary_mtx, auc_thresholds = binarize(auc_mtx.iloc[:,start:start+1], seed=42, num_workers=1)

In [ ]:
# auc_mtx['Klf2'].to_csv(BASE_FOLDER+"error_column.txt")
# auc_mtx = auc_mtx.drop('Klf2', axis=1)

In [ ]:
# Calculate Binary Matrix
binary_mtx, auc_thresholds = binarize(auc_mtx, seed=42, num_workers=1)
binary_mtx.to_csv(BASE_FOLDER+"binary.csv")
binary_mtx